In [1]:
with open('city.txt','r',encoding='utf-8') as fp:
    city_list=fp.readlines()
    city_list=[line.strip().split(' ')[1] for line in city_list]

In [7]:
import random
import json
import time 

prompt_template='''
给定一句话：“%s”，请你按步骤要求工作。

步骤1：识别这句话中的城市和日期共2个信息
步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}

请问，这个JSON字符串是：
'''

samples = []

Q_list=[
    ('{city}{year}年{month}月{day}日的天气','%Y-%m-%d'),
    ('{city}{year}年{month}月{day}号的天气','%Y-%m-%d'),
    ('{city}{month}月{day}日的天气','%m-%d'),
    ('{city}{month}月{day}号的天气','%m-%d'),

    ('{year}年{month}月{day}日{city}的天气','%Y-%m-%d'),
    ('{year}年{month}月{day}号{city}的天气','%Y-%m-%d'),
    ('{month}月{day}日{city}的天气','%m-%d'),
    ('{month}月{day}号{city}的天气','%m-%d'),

    ('你们{year}年{month}月{day}日去{city}玩吗？','%Y-%m-%d'),
    ('你们{year}年{month}月{day}号去{city}玩么？','%Y-%m-%d'),
    ('你们{month}月{day}日去{city}玩吗？','%m-%d'),
    ('你们{month}月{day}号去{city}玩吗？','%m-%d'),
]

# 生成一批"1月2号"、"1月2日"、"2023年1月2号", "2023年1月2日", "2023-02-02", "03-02"之类的话术, 教会它做日期转换
for i in range(1000):
    Q=Q_list[random.randint(0,len(Q_list)-1)]
    city=city_list[random.randint(0,len(city_list)-1)]
    year=random.randint(1990,2025)
    month=random.randint(1,12)
    day=random.randint(1,28)
    time_str='{}-{}-{}'.format(year,month,day)
    date_field=time.strftime(Q[1],time.strptime(time_str,'%Y-%m-%d'))
    Q=Q[0].format(city=city,year=year,month=month,day=day) # 问题
    A=json.dumps({'city':city,'date':date_field},ensure_ascii=False)  # 回答
    samples.append(
        {
            "type": "chatml", 
            "messages": [
                {
                    "role": "user", 
                    "content": Q
                }, 
                {
                    "role": "assistant", 
                    "content": A
                }
            ],
            "source": "self-made"
        }
    )


In [8]:
samples[:3]

[{'type': 'chatml',
  'messages': [{'role': 'user', 'content': '东营市2003年1月25号的天气'},
   {'role': 'assistant', 'content': '{"city": "东营市", "date": "2003-01-25"}'}],
  'source': 'self-made'},
 {'type': 'chatml',
  'messages': [{'role': 'user', 'content': '11月11日白朗县的天气'},
   {'role': 'assistant', 'content': '{"city": "白朗县", "date": "11-11"}'}],
  'source': 'self-made'},
 {'type': 'chatml',
  'messages': [{'role': 'user', 'content': '你们4月3号去阳山县玩吗？'},
   {'role': 'assistant', 'content': '{"city": "阳山县", "date": "04-03"}'}],
  'source': 'self-made'}]

In [10]:
with open('samples.jsonl', 'w') as file:
    for sample in samples:
        file.write(json.dumps(sample) + '\n')